In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import sys
import torchvision.transforms as transforms
import pickle


In [18]:
def log_softmax(vec):
    max_val = np.max(vec)
    exp = np.exp(vec - max_val)
    log_sum_exp = np.log(np.sum(exp))
    return vec - max_val - log_sum_exp


def normalize_array(vec):
    total = sum(vec)
    return np.apply_along_axis(lambda x: x/ total, axis=0, arr = vec)

In [19]:
# Creat and save the model
class LeNet(nn.Module):
    # initial instances creates random weights by default
    def __init__(self):
        super(LeNet, self).__init__()
        linear = nn.Linear
        self.fc1 = linear(784, 300, bias=False)
        self.fc2 = linear(300, 100, bias=False)
        self.fc3 = linear(100, 10, bias=False)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = LeNet().to(device=device)
model.eval()
torch.save(model, 'LeNet.pt')


In [20]:
# Load model and paramaters

# # The weights from the modle created above are random.
# lenet = torch.load('LeNet.pt')
# state_dict = lenet.state_dict()
# fc1_weight = state_dict['fc1.weight'].cpu().numpy()
# fc2_weight = state_dict['fc2.weight'].cpu().numpy()
# fc3_weight = state_dict['fc3.weight'].cpu().numpy()


# Pretrained model from lightning [https://github.com/open-photonics/lightning/tree/8f0804d07d26e58b2855bbc25a3283fd338c1b4e]
fc1_weight = pickle.load(open("/home/rjtomich/photonic_compiler/inference_pratice/LeNet_weights/fc_1.p", "rb"))
fc2_weight = pickle.load(open("/home/rjtomich/photonic_compiler/inference_pratice/LeNet_weights/fc_2.p", "rb"))
fc3_weight = pickle.load(open("/home/rjtomich/photonic_compiler/inference_pratice/LeNet_weights/fc_3.p", "rb"))

print(fc1_weight.shape)
print(fc2_weight.shape)
print(fc3_weight.shape)


(300, 784)
(100, 300)
(10, 100)


In [21]:
# get a test image
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = mnist.load_data()

def random_img():
    image_index = np.random.randint(0, x_test.shape[0])
    image = x_test[image_index]
    label = y_test[image_index]

    # plt.imshow(image, cmap='gray')
    # plt.title('Label: {}'.format(label))
    # plt.show()

    # flatten and normallization
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) ])

    img_tr = transform(image)
    img_flat = torch.flatten(img_tr)
    img_np = np.array(img_flat)
    return img_np, label

In [22]:
def my_inference(img_np):
    l1 = img_np @ fc1_weight.T
    relu_l1 = np.maximum(0, l1) # clips negitives to 0

    l2 = relu_l1 @ fc2_weight.T
    relu_l2 = np.maximum(0, l2)

    l3 = relu_l2 @ fc3_weight.T

    soft_relu_l3 = log_softmax(l3)

    probs = np.exp(soft_relu_l3)
    sum_probs = np.sum(probs)
    return(np.argmax(probs))


In [32]:
right = 0
wrong = 0
for _ in range(10000):
    img, label = random_img()
    if my_inference(img) == label:
        right +=1
    else:
        wrong += 1

print(f'{right} / {wrong+right}')
print(f'accuracy {right/(wrong + right)}')

9746 / 10000
accuracy 0.9746
